In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from groq import Groq

# Initialize the Groq client with your API key (set your actual key here or via an environment variable)
client = Groq(
    api_key="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"   # ← replace with your real key (or load it from os.environ)
)

        Example 2:
        Question: What is the role of reinforcement learning in AI?
        Answer: Reinforcement learning (RL) enables AI agents to learn optimal behaviors through trial and error by maximizing cumulative rewards from environment interactions. It is pivotal in sequential decision-making tasks like game playing (e.g., AlphaGo), robotics, and autonomous systems. RL algorithms iteratively improve policies, balancing exploration and exploitation. Recent advances integrate deep learning for complex environments.

        Example 3:
        Question: How effective are BERT embeddings for document retrieval?
        Answer: BBERT embeddings enhance document retrieval by capturing contextual semantics, outperforming traditional methods like TF-IDF in understanding query-document relevance. However, their effectiveness depends on fine-tuning for specific tasks and computational resources. BERT’s fixed-length inputs can truncate long documents, limiting full-context use. Hybrid approaches (e.g., BERT with BM25) often yield optimal results. Domain adaptation further boosts performance.


In [5]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """
    if strategy == "zero-shot":
        return query 

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute. LSTMs remain useful for low-resource or small-scale tasks.


        Now, answer the question:
        {query}"""
        
    elif strategy == "Role":
        return f"""You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Answer the question:

        {query}"""

    elif strategy == "Motivation (Negative)":
        return f"""
        Answer the given question.
        Failure to produce a grounded answer would lead to irreversible damage to credibility.

        {query}"""

    elif strategy == "Motivation (Positive)":
        return f"""
        Answer the given question.
        Since you have a fervour for well-grounded answers, I trust your ability to do this job well and look forward to reading your response.

        {query}"""

    elif strategy == "Role + Motivation (Negative)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Answer the given question.
        Failure to produce a grounded answer would lead to irreversible damage to credibility.

        {query}"""
    
    elif strategy == "Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Answer the given question.
        Since you have a fervour for well-grounded answers, I trust your ability to do this job well and look forward to reading your response.

        {query}"""

    elif strategy == "Instruction + Role + Motivation (Negative)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Answer the given question.
        Failure to produce a grounded answer would lead to irreversible damage to credibility.
        Use the following steps:
            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""
    
    elif strategy == "Instruction + Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Answer the given question
        Since you have a fervour for well-grounded answers, I trust your ability to do this job well and look forward to reading your response.
        Use the following steps:
            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""
        
    elif strategy == "chain-of-thought (Standard)":
        return f"""Answer the question by first breaking it downstep by step:

            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Neutral)":
        return f"""You are an AI assistant tasked with answering a research question using a set of documents retrieved from a scientific corpus. Your answer must be concise, factually accurate, and include all relevant corpus citations. 
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Praise)":
        return f"""You are a brilliant AI assistant known for generating clear and accurate answers. Take your time to think carefully and provide a precise response.  Use the following steps:

            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Punishment)":
        return f"""You are an AI assistant evaluated on accuracy. Ensure your answer is correct and well thought out. Mistakes reduce trust in your responses and may result in something going horribly wrong. Use the following steps: 

            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Academic Aspiration)":
        return f"""You are a researcher specializing in Computer Science and NLP, aiming for publication-quality clarity and accuracy. Provide a well-supported answer with clear reasoning. Use the following steps:

            1. Identify key elements.
            2. Gather relevant knowledge.
            3. Formulate the answer.

        Now, process this query:
        {query}"""

    elif strategy == "self-verification":
        return f"""Answer the question:
        {query}
        
        Then review your answer and highlight any uncertain points, revising as necessary."""

    elif strategy == "chain-of-verification":
        return f"""Answer the question in four stages:
        1. **Draft**: Provide an initial answer.
        2. **Verify**: List each claim and check its support.
        3. **Revise**: Correct or remove unsupported claims.
        4. **Finalize**: Present the refined answer with notes on verification.
        
        Question: {query}"""

    elif strategy == "chain-of-citation":
        return f"""Answer the question by listing each key claim as a bullet point prefixed with 'Claim:'
 
        Question: {query}"""

    elif strategy == "chain-of-quote":
        return f"""Answer the question step by step, and for each step include a direct quote from your internal knowledge in quotation marks.

        Question: {query}"""

    elif strategy == "Academic Aspiration (2-stage)":
        return f"""You are a rising academic in the field of artificial intelligence. Your work is beginning to attract attention, and every paper you write contributes to your growing reputation. It’s essential that your answers are accurate, well-supported, and ethically cited. Misattributed or missing citations can damage your credibility and compromise the trust of the research community. Cite with care, and write like your future depends on it. 
    Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Punishment 2 stage)":
        return f"""You are an AI assistant whose output will be evaluated for factual accuracy and proper attribution. Providing incomplete, incorrect, or hallucinated citations will result in penalization and reduce the trustworthiness of your response. Only cite sources that are directly relevant and verifiable from the retrieved documents.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Praise 2 stage)":
        return f"""You are an AI assistant known for your high-quality answers and excellent citation accuracy. Citing the correct sources is one of the most important ways you demonstrate trustworthiness, attention to detail, and helpfulness. Providing accurate, complete citations helps users understand and trust your response.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "self-prompt-tuning (role-gen)":
        return f"""You are an expert at role selection for problem solving. Generate a professional role description following this format:

A: This question is a [DOMAIN] problem involving [KEY CONCEPTS]. To better solve it, I will act as a [SPECIFIC ROLE] who [EXPERTISE DESCRIPTION].

Examples:
Q: Can brain cells move? By movement I mean long distance migration (preferably within the brain only).
A: This question is a neuroscience problem involving cell biology and migration. To better solve it, I will act as a neuroscientist who specializes in the study of the brain and its cellular behaviors.

Q: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
A: This question is a computer vision problem involving architectural comparisons. To better solve it, I will act as a deep learning researcher who focuses on model architecture analysis and performance benchmarking.

Now generate a role description for:
Q: {query}"""

    elif strategy == "self-prompt-tuning (answer)":
        # This will be populated dynamically with the generated role
        return f"""{query['generated_role']}

Using this expertise and the retrieved documents below, provide a comprehensive answer:
{query}"""

    elif strategy == "step-back prompting (stage1)":
        return f"""You are an expert research assistant. Paraphrase specific questions into fundamental scientific inquiries:

        Examples:
        Original: How does contrastive learning improve representation learning in vision transformers?
        Stepback: What are the fundamental mechanisms by which self-supervised learning techniques enhance representation learning in transformer architectures?

        Original: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
        Stepback: What architectural differences between convolutional networks and transformer models affect computer vision performance metrics?

        Original: How effective is chain-of-thought prompting for mathematical reasoning in PaLM-2?
        Stepback: What cognitive science principles underlie the effectiveness of decomposition strategies in large language model reasoning?

        Original: What causes gradient instability in 4-bit quantized LLM fine-tuning?
        Stepback: What numerical precision challenges emerge during post-training quantization of neural network parameters?

        Original: How does Med-PaLM 2 achieve state-of-the-art on medical QA benchmarks?
        Stepback: What architectural adaptations and training strategies improve language model performance in domain-specific knowledge tasks?

        Now generate a step-back question for:
        Original: {query}
        Stepback:"""

    elif strategy == "step-back prompting (stage2)":
        return f"""You are an expert at world knowledge. Use both contexts to answer:
        
        Original Context:
        {query['original_context']}
        
        Stepback Context:
        {query['stepback_context']}
        
        Original Question: {query['original_question']}
        """






In [6]:
def generate_plain_answer(query, strategy):
    prompt = reformulate_query(query, strategy)
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [7]:
import os
import pickle
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
from groq import Groq
from sentence_transformers import SentenceTransformer
from gritlm import GritLM

# Configuration
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
GROQ_API_KEY = "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"
os.makedirs(WORKSPACE_PATH, exist_ok=True)

In [8]:
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
E5_INDEX_PATH = os.path.join(WORKSPACE_PATH, "e5_index.pkl")
GTR_INDEX_PATH = os.path.join(WORKSPACE_PATH, "gtr_index.pkl")
GRIT_INDEX_PATH = os.path.join(WORKSPACE_PATH, "grit_index.pkl")

In [9]:
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from groq import Groq
import os
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from groq import Groq
import time
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define workspace path
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"

# Set environment variables to store large files in workspace
os.environ["HF_HOME"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["HF_DATASETS_CACHE"] = os.path.join(WORKSPACE_PATH, "datasets")
os.environ["NLTK_DATA"] = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure necessary directories exist
os.makedirs(os.environ["HF_HOME"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["HF_DATASETS_CACHE"], exist_ok=True)
os.makedirs(os.environ["NLTK_DATA"], exist_ok=True)
BM25_INDEX_PATH = os.path.join(WORKSPACE_PATH, "bm25_index.pkl")
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
NLTK_PATH = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure directory exists
os.makedirs(NLTK_PATH, exist_ok=True)

# Set NLTK data path
nltk.data.path.append(NLTK_PATH)

# Force download punkt & stopwords
nltk.download("punkt", download_dir=NLTK_PATH, force=True)
nltk.download("stopwords", download_dir=NLTK_PATH, force=True)
nltk.download("wordnet", download_dir=NLTK_PATH, force=True)
print(f"Workspace set up at: {WORKSPACE_PATH}")
DATASET_CACHE_DIR = os.path.join(WORKSPACE_PATH, "huggingface_datasets")

# Ensure the directory exists
os.makedirs(DATASET_CACHE_DIR, exist_ok=True)

# Load datasets and store them in workspace cache
query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_s2orc_data = load_dataset("princeton-nlp/LitSearch", "corpus_s2orc", split="full", cache_dir=DATASET_CACHE_DIR)

print("Datasets loaded successfully and stored in:", DATASET_CACHE_DIR)

query_df = query_data.to_pandas()
corpus_clean_df = corpus_clean_data.to_pandas()
corpus_s2orc_df = corpus_s2orc_data.to_pandas()
corpus_df = corpus_clean_df

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws
Datasets loaded successfully and stored in: /data/horse/ws/uskh580e-myws/huggingface_datasets


In [13]:
class E5Retriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("intfloat/e5-large-v2", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save E5 embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing E5 embeddings...")
        texts = ["passage: " + text for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=256, normalize_embeddings=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(E5_INDEX_PATH):
            try:
                with open(E5_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached E5 embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(E5_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode(["query: " + query], normalize_embeddings=True)
        scores = np.dot(query_embedding, self.embeddings.T)
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
e5_retriever = E5Retriever(corpus_df)

if e5_retriever.embeddings is None:
    print("⚠️ E5 embeddings not found, computing...")
    e5_retriever._prepare_embeddings()
class GTRRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("sentence-transformers/gtr-t5-large", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GTR embeddings if not cached."""
        if self._try_load_cache():
            return
        
        print("🔄 Computing GTR embeddings...")
        texts = self.corpus_df["abstract"].astype(str)
        self.embeddings = self.model.encode(texts, batch_size=256, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GTR_INDEX_PATH):
            try:
                with open(GTR_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GTR embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GTR_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode([query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
gtr_retriever = GTRRetriever(corpus_df)

if gtr_retriever.embeddings is None:
    print("⚠️ GTR embeddings not found, computing...")
    gtr_retriever._prepare_embeddings()
class GRITRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = GritLM("GritLM/GritLM-7B", device_map="auto", torch_dtype=torch.bfloat16, mode="embedding")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GRITLM embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing GRIT embeddings...")
        texts = [f"<|embed|>\n{text}" for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GRIT_INDEX_PATH):
            try:
                with open(GRIT_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GRIT embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GRIT_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        formatted_query = f"<|user|>\nRepresent this query for retrieving relevant documents:\n<|embed|>\n{query}"
        query_embedding = self.model.encode([formatted_query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
grit_retriever = GRITRetriever(corpus_df)

if grit_retriever.embeddings is None:
    print("⚠️ GRITLM embeddings not found, computing...")
    grit_retriever._prepare_embeddings()

✅ Loaded cached E5 embeddings
✅ Loaded cached GTR embeddings


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn
✅ Loaded cached GRIT embeddings


In [14]:
attribution_results = []
strategies = ["zero-shot", "few-shot", "chain-of-thought (Standard)","Role","Motivation (Negative)","Motivation (Positive)","Role + Motivation (Negative)","Role + Motivation (Positive)","Instruction + Role + Motivation (Negative)","Instruction + Role + Motivation (Positive)"]

In [11]:
sample_frac = 0.1  # fraction of the dataset to use (e.g., 0.1 = 10%)
total_queries = len(query_df)
query_df = query_df.sample(frac=sample_frac, random_state=42)

each claim separate

In [15]:
# 6. Loop: generate then attribute with claim-level retrieval
k_claim = 3  # ⬅️ adjust this to retrieve more/less per claim

def extract_claims(answer_text):
    # Splits on sentence boundaries and newlines
    parts = re.split(r'[\.!?]\s+|\n', answer_text)
    return [p.strip() for p in parts if p.strip()]

attribution_results = []
for _, row in tqdm(query_df.iterrows(), total=len(query_df), desc="Claim-level attribution"):
    q = row['query']
    # parse ground-truth IDs
    cell = row['corpusids']
    if isinstance(cell, str):
        truth_ids = ast.literal_eval(cell)
    elif isinstance(cell, (list, tuple, np.ndarray, pd.Series)):
        truth_ids = list(cell)
    else:
        truth_ids = []

    for strategy in strategies:
        ans = generate_plain_answer(q, strategy)
        claims = extract_claims(ans)

        # retrieve k_claim docs for each claim
        all_cited = []
        for claim in claims:
            docs = grit_retriever.retrieve(claim, k=k_claim)
            all_cited.extend(docs)

        # dedupe while preserving order
        cited_ids = list(dict.fromkeys(all_cited))

        tp = len(set(cited_ids) & set(truth_ids))
        precision = tp / len(cited_ids) if cited_ids else 0
        recall    = tp / len(truth_ids)    if truth_ids else 0

        attribution_results.append({
            'Query': q,
            'Strategy': strategy,
            'Generated Answer': ans,
            'Claims': claims,
            'Claim Citations': cited_ids,
            'Precision': precision,
            'Recall': recall
        })

# 7. Save & inspect
results_df = pd.DataFrame(attribution_results)
results_df.to_csv('post_gen_claims_k{}.csv'.format(k_claim), index=False)
print(f"Saved claim-level results (k_claim={k_claim}) to post_gen_claims_k{k_claim}.csv")
display(results_df.groupby('Strategy')[['Precision','Recall']].mean())


Claim-level attribution: 100%|████████████████████████████████████████████████████████| 60/60 [1:05:28<00:00, 65.48s/it]


Saved claim-level results (k_claim=3) to post_gen_claims_k3.csv


,Precision,Recall
Strategy,,
Instruction + Role + Motivation (Negative),0.013464,0.591667
Instruction + Role + Motivation (Positive),0.010888,0.533333
Motivation (Negative),0.019303,0.616667
Motivation (Positive),0.019017,0.583333
Role,0.016957,0.508333
Role + Motivation (Negative),0.013326,0.541667
Role + Motivation (Positive),0.013774,0.566667
chain-of-thought (Standard),0.014244,0.558333
few-shot,0.048273,0.533333


In [22]:
results_df = pd.DataFrame(attribution_results)
results_df.to_csv('post_gen_attribution_results.csv', index=False)
print("Results saved to post_gen_attribution_results.csv")
display(results_df.groupby('Strategy')[['Precision','Recall']].mean())

Results saved to post_gen_attribution_results.csv


,Precision,Recall
Strategy,,
chain-of-citation,0.115386,0.551883
chain-of-quote,0.113631,0.559135
chain-of-thought (Academic Aspiration),0.116680,0.582520
chain-of-thought (Praise),0.117082,0.582101
chain-of-thought (Punishment),0.117401,0.588377
chain-of-thought (Standard),0.119600,0.596327
chain-of-verification,0.113914,0.568433
few-shot,0.118873,0.567457
self-verification,0.114146,0.563970


All claims together

In [18]:
for _, row in tqdm(query_df.iterrows(), total=len(query_df), desc="Post-gen attribution"):
    q = row['query']
    # Handle various corpusids types
    cell = row['corpusids']
    if isinstance(cell, str):
        truth_ids = ast.literal_eval(cell)
    elif isinstance(cell, (list, tuple, np.ndarray, pd.Series)):
        truth_ids = list(cell)
    else:
        truth_ids = []

    for strategy in strategies:
        ans = generate_plain_answer(q, strategy)
        cited_ids = grit_retriever.retrieve(ans, k=5)
        tp = len(set(cited_ids) & set(truth_ids))
        precision = tp / len(cited_ids) if cited_ids else 0
        recall = tp / len(truth_ids) if truth_ids else 0
        attribution_results.append({
            'Query': q,
            'Strategy': strategy,
            'Generated Answer': ans,
            'Attributed Corpus IDs': cited_ids,
            'Precision': precision,
            'Recall': recall
        })

# --- 7. Save and display results ---
results_df = pd.DataFrame(attribution_results)
results_df.to_csv('post_gen_attribution_results.csv', index=False)
print("Results saved to post_gen_attribution_results.csv")
display(results_df.groupby('Strategy')[['Precision','Recall']].mean())


Post-gen attribution: 100%|█████████████████████████████████████████████████████████████| 60/60 [09:52<00:00,  9.87s/it]


Results saved to post_gen_attribution_results.csv


,Precision,Recall
Strategy,,
chain-of-citation,0.119026,0.567275
chain-of-quote,0.120244,0.571385
chain-of-thought (Academic Aspiration),0.125419,0.601471
chain-of-thought (Praise),0.124810,0.594165
chain-of-thought (Punishment),0.124810,0.597210
chain-of-thought (Standard),0.127245,0.609691
chain-of-verification,0.122374,0.585337
few-shot,0.122983,0.587316
self-verification,0.120852,0.578945
